# **并行网络GoogleNet**

## **Inception**

<div align=center>
<img width="500" src="../image/5.9_inception.svg"/>
</div>
<div align=center>图5.8 Inception块的结构</div>

GoogleNet由Iception块组成，每个Inception块内有多条并行的线路     
在上图中四条并行线路分别是1*1,3*3,5*5,1*1,中间的两条线路使用了1*1的卷积来减少通道数目     
以上4条线路都使用了合适的padding来保证输出的维度一致      
最后将输出连接形成最终的输出

每条线路的输出通道数可以自行设定

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

In [2]:
import sys
sys.path.append(r'C:\D\ProgramFile\jupyter\torch_learn\dive_to_dp\utils') 
import d2lzh as d2l

In [3]:
device = torch.device('cuda')

In [4]:
class Inception(nn.Module):
    '''
    in_c:输入通道数
    c1-c4:各线路内各层的输出通道数
    '''
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)
        
    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1) # 以NCHW方式排列

## **GoogleNet模型**

GoogleNet在主题卷积部分使用了5个模块，每个模块之间使用了步幅为2的$3 \times 3$最大值池化来减少输出对的高宽。

第一个模块是$7\times7$的卷积层

In [5]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第二个模块有两个卷积层分别是$1\times1$ 64通道的卷积层,$3\times3$ 192通道的卷积层

In [6]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第三个模块串联了两个Inception模块，具体参数见代码.第一个Iception模块的输出通道是256

In [7]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第四个模块串联了5个Inception结构，具体参数如下

In [8]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

第五个模块使用了两个Inception，最后使用了平均值池化的方式把每个通道的长宽变为1

In [9]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())
net = nn.Sequential(b1, b2, b3, b4, b5,
                    d2l.FlattenLayer(),
                    nn.Linear(1024, 10)).cuda()

In [10]:
import torchsummary
torchsummary.summary(net, (1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           3,200
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
            Conv2d-4           [-1, 64, 56, 56]           4,160
            Conv2d-5          [-1, 192, 56, 56]         110,784
         MaxPool2d-6          [-1, 192, 28, 28]               0
            Conv2d-7           [-1, 64, 28, 28]          12,352
            Conv2d-8           [-1, 96, 28, 28]          18,528
            Conv2d-9          [-1, 128, 28, 28]         110,720
           Conv2d-10           [-1, 16, 28, 28]           3,088
           Conv2d-11           [-1, 32, 28, 28]          12,832
        MaxPool2d-12          [-1, 192, 28, 28]               0
           Conv2d-13           [-1, 32, 28, 28]           6,176
        Inception-14          [-1, 256,

In [11]:
batch_size = 64
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)

lr, num_epochs = 0.002, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()

In [12]:
for epoch in range(num_epochs):
    train_l_sum, train_acc_sum, n, batch_count = 0.0, 0.0, 0, 0
    for X, y in train_iter:
        X = X.to(device)
        y = y.to(device)
        y_hat = net(X)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        train_l_sum += l.cpu()
        train_acc_sum += (y_hat.argmax(dim=1) == y).float().sum().cpu()
        n += y.shape[0]
        batch_count += 1
    test_acc = d2l.evaluate_accuracy(test_iter, net)
    print(f'epoch{epoch+1}: loss {train_l_sum/batch_count:.4f} train_acc {train_acc_sum / n:.4f} test_acc {test_acc:.4f}')

epoch1: loss 1.1894 train_acc 0.5409 test_acc 0.7922
epoch2: loss 0.4073 train_acc 0.8486 test_acc 0.8545
epoch3: loss 0.3454 train_acc 0.8699 test_acc 0.8656
epoch4: loss 0.3100 train_acc 0.8832 test_acc 0.8827
epoch5: loss 0.2902 train_acc 0.8902 test_acc 0.8802
